-sandbox
<img width="280" src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>                   

<br>
> # Bronze para Silver

<br>

## 04 - Pedidos de empréstimo 2

Padronizar o formato **`request_amount`** para `Pedidos de Compra` e escrever dois DataFrames em dois diretórios diferentes: 
  - um para `pedidos de empréstimo` sem problemas
  - outro para `pedidos de empréstimo` com valores faltantes ou inversores na lista negra que precisam de verificações manuais executadas pela equipe

-sandbox
<h2 style="color:red">Informações:</h2>

- O objetivo:
   - o que estamos construindo agora: padronizando valores na tabela de `pedidos de empréstimo` e separando linhas para verificação manual
   - por que é útil: às vezes, perder linhas não é viável; os dados corrigidos manualmente serão utilizados mais tarde
- utilizar `UDF`

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Configurações úteis<br>

In [0]:
%run ./Includes/10-ClassroomSetup

Preparing the Python environment...

Defining courseware-specific utility methods...

Defining user-facing utility methods...

Defining custom variables for this lesson...

Initializing Databricks Academy's testing framework...

...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Cleaning up the learning environment... Removed working directory: dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/04_pedidos_emprestimo_2_psp

WARNING: This notebook was tested on DBR 7.0, but we found DBR 7.3. 
 Using an untested DBR may yield unexpected results and/or various errors 
 Please update your cluster configuration and/or download a newer version of this course before proceeding.

The following variables were defined for you: 
 financeDataPath : dbfs:/mnt/training/finance-org 
 The location of the finance dataset used in this capstone project. 
 
 sourceDataPath : dbfs:/mnt/training/finance-org/solutions 
 The location of the specific datasets used in this capstone project as source data. 
 
 silverLoanRequestsPath : dbfs:/mnt/training/finance-org/solutions/silver/loan_requests 
 dbfs:/mnt/training/finance-org/solutions/silver/loan_requests 
 
 manualCheckPath : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/04_pedidos_emprestimo_2_psp/silver/loan_requests_for_manual_check 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/04_pedidos_emprestimo_2_psp/silver/loan_requests_for_manual_check 
 
 targetDirectory : dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/04_pedidos_emprestimo_2_psp/silver/loan_requests_cleansed.delta 
 dbfs:/user/thomaz@thomazrossito.com.br/capstone_finance/04_pedidos_emprestimo_2_psp/silver/loan_requests_cleansed.delta 
 The following database were created for you: 
 Now using the database identified by databaseName : 
 thomaz_thomazrossito_com_br_capstone_finance_04_pedidos_emprestimo_2_psp 
 This is a private, per-notebook, database used to provide isolation from other users and exercises. 
 All done!

All done!

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Etapas a serem concluídas: <br>

<br>
- Ler as solicitações de empréstimo a partir de  **`silverLoanRequestsPath`** para um DataFrame
- Crie uma função que extrai a string de moeda de do campo **`request_amount`** e converte `$` em `USD`
- Registre esta função como `UDF` e coloque o nome de **`get_currency_udf`**
- Verificar se é possível conseguir o mesmo resultado com funções integradas
  - Crie um novo DataFrame onde será usado as funções built-in para extrair tanto a **`moeda`** quanto a **`quantidade`** da coluna **`valor_de_pedida`**
  - coloque o nome dessas novas colunas de **`moeda`** e **`montante`**
- Delete a coluna **`request_amount`** assim que terminar as extrações
- Crie um novo DataFrame chamado **`finalOK`**, que contém aqueles valores onde as colunas **`missing_amount`** e **`banned_investor`** são falsas
- Salve o DataFrame `finalOK` como uma tabela Delta em **`targetDirectory`**
- Crie um novo DataFrame chamado **`finalCheck`** contendo aqueles valores onde as colunas **`missing_amount`** ou **`banned_investor`** são verdadeiras
- Salve o DataFrame `finalCheck` como uma tabela Delta em **`manualCheckPath`**
- Retorne o `UDF`, `finalOK` e `finalCheck` como uma saída da função

<p> 
  O esquema do DataFrame deve ser:
  
|name|type|
|---|---|
|investor_id|LongType|
|loan_id|LongType|
|product_id|LongType|
|request_length|LongType|
|request_time|TimestampType|
|valid_to|TimestampType|
|password_hash|StringType|
|missing_amount|BooleanType|
|banned_investor|BooleanType|
|loan_alert|StringType|
|currency|StringType|
|amount|IntegerType|

In [0]:
def func_pedidoEmprestimo(spark, silverLoanRequestsPath, manualCheckPath, targetDirectory):

  
  from pyspark.sql.functions import col, regexp_extract, when
  import re
  
  
  # Leia as solicitações de empréstimo a partir de `silverLoanRequestsPath`
  df = (spark.read
             .format("delta")
             .load(silverLoanRequestsPath))
  
  
  # Cria uma função, que extrai a string de moeda do campo `request_amount` e converte de `$` em `USD`
  def get_currency(str):
      if not str:
         return ""
      moedaStr = re.sub("[0-9 ]+","",str)
      return moedaStr if moedaStr != "$" else "USD"

  # Registra essa função como UDF com o nome `get_currency_udf`
  get_currency_udf = udf(get_currency)
  
  
  # Agora que foi implementado uma UDF, veremos se é possível obter o
  # mesmo com funções built-in. Crie um novo DataFrame que usa 
  # funções integradas para extrair tanto a `moeda` quanto a `quantia` da coluna `valor_de_pedida`.
  # Chame essas novas colunas de `moeda` e` quantia`.
  # Delete a coluna `request_amount` quando terminar as extrações.
  preco_moeda_raw = regexp_extract(col("request_amount"),"^([^0-9 ]+)", 0)
  preco_moeda_col = when(preco_moeda_raw == "$", "USD").otherwise(preco_moeda_raw).alias("currency")

  preco_quatia_col = regexp_extract(col("request_amount"),"([0-9]+)$", 0).cast("integer").alias("amount")
  
  finalDF = (df.select("*", preco_moeda_col, preco_quatia_col)
               .drop("request_amount"))

  
  # Crie um novo DataFrame chamado de "finalOK", que contém os valores onde a
  # coluna "missing_amount" e as colunas "banned_investor" são falsas.
  finalOK = (finalDF.filter(col("missing_amount") == False)
                    .filter(col("banned_investor") == False))
  
  
  # Salvando o dataframe "finalOK" em uma tabela Delta em "targetDirectory".
  finalOK.write \
         .mode("overwrite") \
         .format("delta") \
         .option("overwriteSchema", "true") \
         .save(targetDirectory) 
          

  # Crie um novo DataFrame com o nome "finalCheck" que contém os valores 
  # onde as colunas "missing_amount" ou "banned_investor" são verdadeiras
  finalCheck = (finalDF.filter((col("missing_amount") == True) | 
                               (col("banned_investor") == True)))
  
  
  # Save finalCheck as a Delta table to `manualCheckPath` with overwrite mode and overwrite schema option.
  finalCheck.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .save(manualCheckPath)
  
  
  # Retorna a UDF, o dataframe finalOK e o o dataframe finalCheck em tuple.
  return (get_currency_udf, finalOK, finalCheck)

In [0]:
(get_currency_udf, finalOK, finalCheck) = func_pedidoEmprestimo(spark, silverLoanRequestsPath, manualCheckPath, targetDirectory)

In [0]:
finalOK.show(2)
finalCheck.show(2)

+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
investor_id| loan_id|product_id|request_length| request_time| valid_to| password_hash|missing_amount|banned_investor|loan_alert|currency|amount|
+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
 747|503830834| 1| 45|2019-09-05 00:24:40|2019-09-20 00:24:40|73ce6846caf83bab7...| false| false| green| USD| 11918|
 817|503830835| 1| 222|2019-09-18 00:01:03|2019-10-03 00:01:03|e4d3baca1bb213f79...| false| false| green| USD| 10408|
+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
only showing top 2 rows

+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
investor_id| loan_id|product_id|request_length| request_time| valid_to| password_hash|missing_amount|banned_investor|loan_alert|currency|amount|
+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
 1137|503830887| 1| 239|2019-09-16 15:49:02|2019-10-01 15:49:02|bc00327126dfb41e0...| true| false| green| null| null|
 220|503830928| 1| 76|2019-09-11 17:16:54|2019-09-26 17:16:54|c152e0e531fb6a44b...| true| false| green| null| null|
+-----------+---------+----------+--------------+-------------------+-------------------+--------------------+--------------+---------------+----------+--------+------+
only showing top 2 rows

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Validação</h2>

In [0]:
realityCheck(func_pedidoEmprestimo, spark, silverLoanRequestsPath, manualCheckPath, targetDirectory)

Points,Test,Result
1,Returns DataFrame with `currency` and without the `request_amount` column,
1,Returns DataFrame with correct `currency` values,
1,UDF returns correct values,
1,Returns correct schema: finalOK,
1,Returns correct schema: finalCheck,
1,Returns DataFrame with correct results: finalOK,
1,Returns DataFrame with correct results: finalCheck,
1,Delta table in place: finalOK,
1,Delta table has correct content: finalOK,
1,Delta table in place: finalCheck,


## <img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Próximo <p>

> * [Transações de estoque]($./05-Transacoes estoque)

-sandbox

<a href="http://www.apache.org/">Apache Software Foundation</a> <p>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a>